In [2]:
import os
import shutil
import random
import torchaudio
import torch
import numpy as np
from pydub import AudioSegment
from tqdm import tqdm

In [2]:
# Define the root folder path
root_folder_path = "C:\\Users\\User\\Apviza\\Project 6\\Output"
output_folder = "C:\\Users\\User\\Apviza\\Project 6\\Model Data\\Altered"

# Initialize an empty list to store dataset dictionaries
dataset_dict_list = []

for root, _, _ in os.walk(root_folder_path):
    for subfolder in os.listdir(root):
        subfolder_path = os.path.join(root, subfolder)
        if os.path.isdir(subfolder_path):
            for sub_root, _, files in os.walk(subfolder_path):
                for file in files:
                    if file.endswith(".wav"):
                        audio_path = os.path.join(sub_root, file)
                        path_components = sub_root.split(os.path.sep)
                        dialect_region = path_components[-2]
                        speaker_id = path_components[-1]
                        dataset_dict = {
                            "audio": audio_path,
                            "dialect_region": dialect_region,
                            "speaker_id": speaker_id,
                            "audio_file": file
                        }
                        dataset_dict_list.append(dataset_dict)

In [3]:
for item in dataset_dict_list:
    audio_file = item["audio_file"]
    speaker_id = item["speaker_id"]
    new_audio_name = f"{speaker_id}_{audio_file}"
    new_audio_path = os.path.join(output_folder, new_audio_name)
    
    # Copy the audio file to the new path with the new name
    shutil.copy(item["audio"], new_audio_path)

In [4]:
# Define paths to the Altered and Pristine folders
altered_folder = "C:\\Users\\User\\Apviza\\Project 6\\Model Data\\Altered"
pristine_folder = "C:\\Users\\User\\Apviza\\Project 6\\Model Data\\Pristine"

# Define paths for the train, test, and validation folders
train_folder = "C:\\Users\\User\\Apviza\\Project 6\\Model Data\\Train"
test_folder = "C:\\Users\\User\\Apviza\\Project 6\\Model Data\\Test"
val_folder = "C:\\Users\\User\\Apviza\\Project 6\\Model Data\\Validation"

In [6]:
# Create train, test, and validation folders if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

# Define the percentage split for train, test, and validation
train_split = 0.7  # 70% for training
test_split = 0.15  # 15% for testing
val_split = 0.15  # 15% for validation

# Iterate through the Altered folder
for filename in os.listdir(altered_folder):
    file_path = os.path.join(altered_folder, filename)
    
    # Randomly assign files to train, test, or validation based on the defined splits
    rand = random.random()
    if rand < train_split:
        shutil.copy(file_path, os.path.join(train_folder, filename))
    elif rand < train_split + test_split:
        shutil.copy(file_path, os.path.join(test_folder, filename))
    else:
        shutil.copy(file_path, os.path.join(val_folder, filename))

# Iterate through the Pristine folder
for filename in os.listdir(pristine_folder):
    file_path = os.path.join(pristine_folder, filename)
    
    # Randomly assign files to train, test, or validation based on the defined splits
    rand = random.random()
    if rand < train_split:
        shutil.copy(file_path, os.path.join(train_folder, filename))
    elif rand < train_split + test_split:
        shutil.copy(file_path, os.path.join(test_folder, filename))
    else:
        shutil.copy(file_path, os.path.join(val_folder, filename))

print("Data has been split into train, test, and validation sets.")

Data has been split into train, test, and validation sets.


In [8]:
output_root = "C:\\Users\\User\\Apviza\\Project 6\\Model Data"  # Change to your output folder path
padded_root = "C:\\Users\\User\\Apviza\\Project 6\\Padded"  # New Padded folder
subfolders = ["train", "test", "validation"]
1859328


1859328

In [9]:
# Define paths
output_root = "C:\\Users\\User\\Apviza\\Project 6\\Model Data"  # Change to your output folder path
subfolders = ["train", "test", "validation"]

# Iterate through train, test, and validation subfolders
for subfolder in subfolders:
    src_folder = os.path.join(output_root, subfolder)
    dest_folder = os.path.join(output_root, subfolder)

    audio_files = [file for file in os.listdir(src_folder) if file.endswith(".mp3")]

    for file in tqdm(audio_files, desc=f"Converting MP3 to WAV for {src_folder}"):
        src_file = os.path.join(src_folder, file)
        dest_file = os.path.join(dest_folder, file.replace(".mp3", ".wav"))

        # Convert .mp3 to .wav
        audio = AudioSegment.from_mp3(src_file)
        audio.export(dest_file, format="wav")

print("MP3 to WAV conversion completed.")

Converting MP3 to WAV for C:\Users\User\Apviza\Project 6\Model Data\train: 100%|██████████| 31013/31013 [1:35:53<00:00,  5.39it/s]
Converting MP3 to WAV for C:\Users\User\Apviza\Project 6\Model Data\test: 100%|██████████| 6598/6598 [22:15<00:00,  4.94it/s]
Converting MP3 to WAV for C:\Users\User\Apviza\Project 6\Model Data\validation: 100%|██████████| 6424/6424 [21:34<00:00,  4.96it/s]

MP3 to WAV conversion completed.


In [3]:
# Define paths
output_root = "C:\\Users\\User\\Apviza\\Project 6\\Model Data"  # Change to your output folder path
padded_root = "C:\\Users\\User\\Apviza\\Project 6\\Truncated"  # New Padded folder
subfolders = ["train", "test", "validation"]

# Create the Padded folder and subfolders
os.makedirs(padded_root, exist_ok=True)
for subfolder in subfolders:
    os.makedirs(os.path.join(padded_root, subfolder), exist_ok=True)

def resample_audio(waveform, target_sample_rate=16000):
    audio = torchaudio.transforms.Resample(
        orig_freq=waveform.shape[1], new_freq=target_sample_rate
    )(waveform)
    return audio

# Define a function to pad and copy audio files
def pad_and_copy_audio(src_folder, dest_folder, max_length):
    audio_files = [file for file in os.listdir(src_folder) if file.endswith(".wav")]

    for file in tqdm(audio_files, desc=f"Processing {src_folder}"):
        src_file = os.path.join(src_folder, file)
        dest_file = os.path.join(dest_folder, file)

        # Check if the destination file already exists, and skip if it does
        if os.path.exists(dest_file):
            continue

        waveform, sample_rate = torchaudio.load(src_file)

        # Pad the waveform if needed
        pad_length = max_length - waveform.size(1)
        if pad_length > 0:
            padding = torch.from_numpy(np.zeros((1, pad_length), dtype=np.float32))
            padded_waveform = torch.cat((waveform, padding), dim=1)
            torchaudio.save(dest_file, padded_waveform, sample_rate)
        else:
            torchaudio.save(dest_file, waveform, sample_rate)

# Iterate through train, test, and validation subfolders
for subfolder in subfolders:
    src_folder = os.path.join(output_root, subfolder)
    dest_folder = os.path.join(padded_root, subfolder)

    # Determine the maximum waveform length in this dataset
    max_waveform_length = 23296

    # Pad and copy the audio files
    pad_and_copy_audio(src_folder, dest_folder, max_waveform_length)

print("Audio padding and copying completed.")

Processing C:\Users\User\Apviza\Project 6\Model Data\train:  51%|█████     | 16957/33212 [00:01<00:01, 8522.38it/s]


LibsndfileError: System error.

In [4]:
output_root = "C:\\Users\\User\\Apviza\\Project 6\\Model Data"
subfolders = ["train", "test", "validation"]

# Initialize a variable to store the smallest waveform length
smallest_waveform_length = float('inf')

for subfolder in subfolders:
    folder_path = os.path.join(output_root, subfolder)
    audio_files = [file for file in os.listdir(folder_path) if file.endswith(".wav")]

    for file in audio_files:
        file_path = os.path.join(folder_path, file)
        waveform, _ = torchaudio.load(file_path)
        waveform_length = waveform.size(1)
        
        if waveform_length < smallest_waveform_length:
            smallest_waveform_length = waveform_length

print("Smallest waveform length:", smallest_waveform_length)

Smallest waveform length: 23296


In [5]:
output_root = "C:\\Users\\User\\Apviza\\Project 6\\Model Data"  # Change to your output folder path
truncated_root = "C:\\Users\\User\\Apviza\\Project 6\\Truncated"  # New Truncated folder
subfolders = ["train", "test", "validation"]

# Create the Truncated folder and subfolders
os.makedirs(truncated_root, exist_ok=True)
for subfolder in subfolders:
    os.makedirs(os.path.join(truncated_root, subfolder), exist_ok=True)

# Define a function to truncate audio files
def truncate_audio(src_folder, dest_folder, max_length):
    audio_files = [file for file in os.listdir(src_folder) if file.endswith(".wav")]

    for file in tqdm(audio_files, desc=f"Processing {src_folder}"):
        src_file = os.path.join(src_folder, file)
        dest_file = os.path.join(dest_folder, file)

        # Check if the destination file already exists, and skip if it does
        if os.path.exists(dest_file):
            continue

        waveform, sample_rate = torchaudio.load(src_file)

        # Truncate the waveform if needed
        if waveform.size(1) > max_length:
            truncated_waveform = waveform[:, :max_length]
            torchaudio.save(dest_file, truncated_waveform, sample_rate)
        else:
            torchaudio.save(dest_file, waveform, sample_rate)

# Iterate through train, test, and validation subfolders
for subfolder in subfolders:
    src_folder = os.path.join(output_root, subfolder)
    dest_folder = os.path.join(truncated_root, subfolder)

    # Determine the maximum waveform length in this dataset
    max_waveform_length = 23296

    # Truncate and copy the audio files
    truncate_audio(src_folder, dest_folder, max_waveform_length)

print("Audio truncating and copying completed.")

Processing C:\Users\User\Apviza\Project 6\Model Data\train: 100%|██████████| 33212/33212 [02:58<00:00, 186.32it/s]
Processing C:\Users\User\Apviza\Project 6\Model Data\test: 100%|██████████| 7057/7057 [00:37<00:00, 186.07it/s]
Processing C:\Users\User\Apviza\Project 6\Model Data\validation: 100%|██████████| 6892/6892 [00:38<00:00, 180.21it/s]

Audio truncating and copying completed.
